In [138]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd

### Apply pre-trained model

In [127]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [139]:
df = pd.read_csv('../sources/data.csv')
df

,Datetime,Symbol,Text,Retweeted,price,30_days_price,90_days_price,180_days_price
0,2023-01-25 03:05:45+00:00,MMM,To me the worst quarter of the season so far i...,NaN,111.454300,107.800003,NaN,NaN
1,2021-11-30 14:22:02+00:00,MMM,"$MRNA, $SQ, $F, $MMM &amp; more… all covered i...",NaN,160.991837,168.187424,142.100510,144.154526
2,2020-09-24 08:36:51+00:00,MMM,If you look at the https://t.co/NS3syOn64o you...,NaN,145.948441,151.170639,159.663681,174.247040
3,2020-08-27 08:46:42+00:00,MMM,thank you to Mike Roman and the great folks at...,NaN,149.278290,147.076584,162.535919,162.996353
4,2020-01-28 17:44:08+00:00,MMM,the stock needs to see that return.. as this i...,NaN,146.491455,134.052170,137.167786,147.174454
...,...,...,...,...,...,...,...,...
12228,2015-05-05 09:52:02+00:00,ZTS,"When you have $DIS, $CSCO and $EOG, $ZTS you a...",NaN,43.352589,46.408531,46.009922,42.058086
12229,2015-05-05 07:49:28+00:00,ZTS,"Going into overdrive for $CSCO, $DIS, $ZTS, an...",NaN,43.352577,46.408527,46.009914,42.058098
12230,2014-06-09 19:57:42+00:00,ZTS,Is $ATHN a $1000 stock? Is $ZTS the enemy of $...,NaN,30.144522,30.568308,34.238846,41.699692
12231,2013-12-03 01:45:23+00:00,ZTS,"Must Buy, $ZTS RT @codybarbo: ""Americans spend...",NaN,29.139360,30.329683,29.112482,29.118053


In [142]:
def get_sentiment(row):
    print(row['Symbol'])
    
    tokens = tokenizer.encode(row['Text'], return_tensors='pt')
    result = model(tokens)

    return int(torch.argmax(result.logits))+1

In [147]:
df['sentiment'] = df.apply(lambda row: get_sentiment(row), axis=1)

MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
MMM
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABT
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ABBV
ACN
ACN
ACN
ACN
ACN
ACN
ACN
ACN
ACN
ACN
ACN
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ATVI
ADM
ADM
ADM
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADBE
ADP
ADP
ADP
ADP
ADP
ADP
ADP
AAP
AAP
AAP
AAP
A
A
APD
APD
APD
APD
AKAM
AKAM
AKAM
AKAM
AKAM
AKAM
AKAM
AKAM
AKAM
ALK
ALB
ALGN
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOGL
GOOG

### Compute the correlation

In [151]:
df['30_days_return'] =	(df['30_days_price'] - df['price']) / df['price']
df['90_days_return'] =	(df['90_days_price'] - df['price']) / df['price']
df['180_days_return'] =	(df['180_days_price'] - df['price']) / df['price']

In [159]:
df_dropna_30 = df.dropna(subset=['30_days_return'])
df_dropna_90 = df.dropna(subset=['90_days_return'])
df_dropna_180 = df.dropna(subset=['180_days_return'])
corr1 = np.corrcoef(df_dropna_30['30_days_return'], df_dropna_30['sentiment'])
corr2 = np.corrcoef(df_dropna_90['90_days_return'], df_dropna_90['sentiment'])
corr3 = np.corrcoef(df_dropna_180['180_days_return'], df_dropna_180['sentiment'])
print(corr1)
print(corr2)
print(corr3)

[[1.         0.00231467]
 [0.00231467 1.        ]]
[[1.         0.00625331]
 [0.00625331 1.        ]]
[[ 1.         -0.00338364]
 [-0.00338364  1.        ]]


In [166]:
df.to_csv('../sources/data.csv', index=False)

In [167]:
df = pd.read_csv('../sources/data.csv')
df

,Datetime,Symbol,Text,Retweeted,price,30_days_price,90_days_price,180_days_price,sentiment,30_days_return,90_days_return,180_days_return
0,2023-01-25 03:05:45+00:00,MMM,To me the worst quarter of the season so far i...,NaN,111.454300,107.800003,NaN,NaN,1,-0.032787,NaN,NaN
1,2021-11-30 14:22:02+00:00,MMM,"$MRNA, $SQ, $F, $MMM &amp; more… all covered i...",NaN,160.991837,168.187424,142.100510,144.154526,1,0.044695,-0.117343,-0.104585
2,2020-09-24 08:36:51+00:00,MMM,If you look at the https://t.co/NS3syOn64o you...,NaN,145.948441,151.170639,159.663681,174.247040,5,0.035781,0.093973,0.193894
3,2020-08-27 08:46:42+00:00,MMM,thank you to Mike Roman and the great folks at...,NaN,149.278290,147.076584,162.535919,162.996353,5,-0.014749,0.088812,0.091896
4,2020-01-28 17:44:08+00:00,MMM,the stock needs to see that return.. as this i...,NaN,146.491455,134.052170,137.167786,147.174454,1,-0.084915,-0.063647,0.004662
...,...,...,...,...,...,...,...,...,...,...,...,...
12228,2015-05-05 09:52:02+00:00,ZTS,"When you have $DIS, $CSCO and $EOG, $ZTS you a...",NaN,43.352589,46.408531,46.009922,42.058086,1,0.070490,0.061296,-0.029860
12229,2015-05-05 07:49:28+00:00,ZTS,"Going into overdrive for $CSCO, $DIS, $ZTS, an...",NaN,43.352577,46.408527,46.009914,42.058098,4,0.070491,0.061296,-0.029859
12230,2014-06-09 19:57:42+00:00,ZTS,Is $ATHN a $1000 stock? Is $ZTS the enemy of $...,NaN,30.144522,30.568308,34.238846,41.699692,1,0.014058,0.135823,0.383326
12231,2013-12-03 01:45:23+00:00,ZTS,"Must Buy, $ZTS RT @codybarbo: ""Americans spend...",NaN,29.139360,30.329683,29.112482,29.118053,5,0.040849,-0.000922,-0.000731


In [126]:
from transformers import BertTokenizer, BertForTokenClassification

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = BertForTokenClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Define a sample sentence to analyze
sentence = 'To me the best quarter of the season so far is $MMM. Just dismal with a ""dry"" January included'

# Tokenize the sentence
inputs = tokenizer.encode(sentence, return_tensors='pt')

# Pass the tokenized input through the model to get the sentiment for each word
outputs = model(inputs)
sentiments = outputs.logits.softmax(dim=-1).tolist()[0]

# Get the sentiment label for each word
sentiment_labels = ['positive', 'slightly positive', 'neutral', 'slightly negative', 'negative']
words = tokenizer.convert_ids_to_tokens(inputs[0])
for word, sentiment in zip(words, sentiments):
    index = sentiment.index(max(sentiment))
    # print(word, sentiment_labels[index], max(sentiment))
    print(f"The sentiment of '{word}' is {sentiment_labels[index]} with a confidence score of {max(sentiment):.2f}")


The sentiment of '[CLS]' is neutral with a confidence score of 0.24
The sentiment of 'to' is slightly positive with a confidence score of 0.26
The sentiment of 'me' is negative with a confidence score of 0.26
The sentiment of 'the' is slightly positive with a confidence score of 0.24
The sentiment of 'best' is neutral with a confidence score of 0.27
The sentiment of 'quarter' is negative with a confidence score of 0.28
The sentiment of 'of' is negative with a confidence score of 0.28
The sentiment of 'the' is negative with a confidence score of 0.30
The sentiment of 'season' is negative with a confidence score of 0.35
The sentiment of 'so' is slightly positive with a confidence score of 0.28
The sentiment of 'far' is slightly positive with a confidence score of 0.27
The sentiment of 'is' is neutral with a confidence score of 0.28
The sentiment of '$' is neutral with a confidence score of 0.25
The sentiment of 'mm' is slightly positive with a confidence score of 0.25
The sentiment of '#